<a href="https://colab.research.google.com/github/dlgur1994/Detecting-and-Classifying-Car-Damage/blob/main/mkkim/dataloader.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
!cp /content/gdrive/MyDrive/KIM123/dataset_accida_segmentation_dataset_v1.zip /content/
! unzip  -qq /content/dataset_accida_segmentation_dataset_v1.zip

# 데이터 살펴보기 

In [4]:
# Torch
import torch
from torch.utils.data import Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor
from torchvision.io import read_image

# Helper libraries
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow_datasets as tfds
from PIL import Image

In [5]:
root = '/content/accida_segmentation_dataset_v1/dent/train'
imgs = list(sorted(os.listdir(os.path.join(root, "images"))))


In [9]:
class SocarDataset(torch.utils.data.Dataset):

    def __init__(self, root, class,transforms=None):
        self.root = root
        self.transforms = transforms
        # 모든 이미지 파일들을 읽고, 정렬하여
        # 이미지와 분할 마스크 정렬을 확인합니다
        self.imgs = list(sorted(os.listdir(os.path.join(root, "images"))))
        self.masks = list(sorted(os.listdir(os.path.join(root, "masks"))))



    


    def __getitem__(self, idx):
        img_path = os.path.join(self.root, "images", self.imgs[idx])
        mask_path = os.path.join(self.root, "masks", self.masks[idx])
        img = Image.open(img_path).convert("RGB")
        # 분할 마스크는 RGB로 변환하지 않음
        # 각 색상은 다른 인스턴스에 해당하며, 0은 배경에 해당 -> 근데 우리데이터는 흰검 밖에 없어서,,
        mask = Image.open(mask_path)
        # numpy 배열을 PIL 이미지로 변환합니다
        mask = np.array(mask)
        # 인스턴스들은 다른 색들로 인코딩 되어 있습니다.
        obj_ids = np.unique(mask)
        # 첫번째 id 는 배경이라 제거합니다
        #obj_ids = obj_ids[1:]
        

        # 컬러 인코딩된 마스크를 바이너리 마스크 세트로 나눕니다
        masks = mask == obj_ids[:, None, None]

        # 각 마스크의 바운딩 박스 좌표를 얻습니다
        num_objs = len(obj_ids)
        print(num_objs)
        boxes = []
        for i in range(num_objs):
            pos = np.where(masks[i])
            xmin = np.min(pos[1])
            xmax = np.max(pos[1])
            ymin = np.min(pos[0])
            ymax = np.max(pos[0])
            boxes.append([xmin, ymin, xmax, ymax])

        # 모든 것을 torch.Tensor 타입으로 변환합니다
        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        # 객체 종류는 한 종류만 존재합니다(역자주: 예제에서는 사람만이 대상입니다)
        labels = torch.ones((num_objs,), dtype=torch.int64)
        masks = torch.as_tensor(masks, dtype=torch.uint8)

        image_id = torch.tensor([idx])
        area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])
        # 모든 인스턴스는 군중(crowd) 상태가 아님을 가정합니다
        iscrowd = torch.zeros((num_objs,), dtype=torch.int64)

        target = {}
        target["boxes"] = boxes
        target["labels"] = labels
        target["masks"] = masks
        target["image_id"] = image_id
        target["area"] = area
        target["iscrowd"] = iscrowd

        if self.transforms is not None:
            img, target = self.transforms(img, target)

        return img, target


    def __len__(self):
        return len(self.imgs)
    
    # Dent/Scratch/Spacing - 지금 코드에서는 안필요함
    def get_label(data_path_list):
        label_list = []
        for path in data_path_list:
            # 앞에서 3번째가 class
            label_list.append(path.split('/')[3])
        return label_list


In [23]:
mask_path = os.path.join("/content/accida_segmentation_dataset_v1/dent/train", "masks", "20190223_11917_20255505_7f990d082050b9753fc4810c7bf6e1e4.jpg")
testmask = Image.open(mask_path)
testmask = np.array(testmask)
obj_ids = np.where(testmask!=0,testmask)
obj_ids

ValueError: ignored

## Dent

In [10]:
img_path = "/content/accida_segmentation_dataset_v1/dent/"

train = SocarDataset(
        root=img_path+"train",
        transforms=None
    )

test = SocarDataset(
        root=img_path+"test",
        transforms=None
    )

valid = SocarDataset(root=img_path+"valid",
        transforms=None
    )


trainloader = torch.utils.data.DataLoader(
    train,
    batch_size=4,
    shuffle=False
)

testloader = torch.utils.data.DataLoader(
    test,
    batch_size=4,
    shuffle=False
)

validloader = torch.utils.data.DataLoader(
    valid,
    batch_size=4,
    shuffle=False
)

print('train 이미지들의 개수: ', len(train))
print('test 이미지들의 개수: ',  len(test))
print('valid 이미지들의 개수: ',  len(valid))

train 이미지들의 개수:  2132
test 이미지들의 개수:  267
valid 이미지들의 개수:  267


In [12]:
train.masks[11]

'20180101_10778_10632865_f61d4ae8ecaa12e9e58411e24d7d0898_augmented2.jpg'